In [2]:
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [5]:
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding

OptimumEmbedding.create_and_save_optimum_model(
    "sentence-transformers/all-MiniLM-L6-v2", "./minilml6v2_onnx"
)

Saved optimum model to ./minilml6v2_onnx. Use it with `embed_model = OptimumEmbedding(folder_name='./minilml6v2_onnx')`.


In [3]:
def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

def subtract(a: float, b: float) -> float:
    """Subtract two numbers and returns the difference"""
    return a - b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

def divide(a: float, b: float) -> float:
    """Divide two numbers and returns the quotient"""
    return a / b

add_tool = FunctionTool.from_defaults(fn=add, name="add", description="Add two numbers")
subtract_tool = FunctionTool.from_defaults(fn=subtract, name="subtract", description="Subtract two numbers")
multiply_tool = FunctionTool.from_defaults(fn=multiply, name="multiply", description="Multiply two numbers")
divide_tool = FunctionTool.from_defaults(fn=divide, name="divide", description="Divide two numbers")

In [6]:
Settings.llm = Ollama(
    model="llama3.2",
    request_timeout=3000.0,
)
Settings.embed_model = OptimumEmbedding(folder_name="./minilml6v2_onnx")
# Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [7]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [8]:
from llama_index.core.tools import QueryEngineTool

budget_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="alarmas_hexing_buget",
    description="A RAG engine with some basic info about Alarm hexing, data in spanish",
)

In [10]:
agent = ReActAgent.from_tools(
    [budget_tool], 
    llm=Settings.llm, 
    verbose=True
)


In [11]:
response = agent.chat("I need information about alarm Voltage Unbalance Start")
print(response)

> Running step ce24caca-830d-46bb-af88-2c8b84f59bf8. Step input: I need information about alarm Voltage Unbalance Start
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: alarmas_hexing_buget
Action Input: {'input': AttributedDict([('title', 'Input'), ('type', 'string')]), 'num_beams': 5}
Observation: Error: 2 validation errors for QueryStartEvent
query.str
  Input should be a valid string [type=string_type, input_value=AttributedDict([('title',...'), ('type', 'string')]), input_type=AttributedDict]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
query.QueryBundle.query_str
  Field required [type=missing, input_value=AttributedDict([('title',...'), ('type', 'string')]), input_type=AttributedDict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
> Running step a954db3e-b6f6-4da3-a19f-0abefa8ffb60. Step input: None
Thought: The user provided an error message indicat

ValueError: Reached max iterations.